In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 读取CSV文件
df = pd.read_csv('he.csv', header=None)

# 获取第一列数据并转为一维数组
arr = df.iloc[:2048,2].values.flatten()
xray = df.iloc[:2048,3].values.flatten()
arr1 = df.iloc[:650,0].values.flatten()
nir= df.iloc[:650,1].values.flatten()

# 归一化处理
arr_normalized = (arr - arr.min()) / (arr.max() - arr.min())
arr1_normalized =( (arr1 - arr1.min()) / (arr1.max() - arr1.min()))+1
xray_normalized = (xray - xray.min()) / (xray.max() - xray.min())
nir_normalized = (nir - nir.min()) / (nir.max() - nir.min())

# 将 arr 和 arr1 拼接成一个数组 X_axis，将 xray 和 nir 拼接成一个数组 Y_axis
X_axis = np.concatenate((arr_normalized, arr1_normalized))
Y_axis = np.concatenate((xray_normalized, nir_normalized))

# 写入CSV文件
data = {'X axis': X_axis, 'Y axis': Y_axis}
df_new = pd.DataFrame(data)
df_new.to_csv('output.csv', index=False)




In [ ]:

import numpy as np

# 定义标准样品的浓度和强度
std_conc = np.array([[1.0, 2.0, 4.0], [100, 200, 400]])
# 定义待测样品的浓度和强度
unknown_conc = np.array([[2.0, 4.0, 8.0], [150, 250, 450]])

# 迭代求解
max_iterations = 10  # 最大迭代次数
tolerance = 0.01  # 收敛条件
n_compounds = std_conc.shape[1]  # 组分数目
n_samples = unknown_conc.shape[0]  # 样品数目

# 初始化假设标准
hyp_std_conc = np.ones(n_compounds)
hyp_std_conc /= np.sum(hyp_std_conc)

# 开始迭代
for i in range(max_iterations):
    # 计算理论强度
    hyp_intensity = np.dot(std_conc.T, hyp_std_conc)
    # 计算偏差
    residual = unknown_conc[1, :] - hyp_intensity
    # 计算灵敏度矩阵
    sens_matrix = std_conc / hyp_intensity
    # 计算权重矩阵
    weights = np.diag(hyp_std_conc)
    # 计算参数变化量
    delta = np.dot(np.dot(np.linalg.inv(np.dot(np.dot(sens_matrix.T, weights), sens_matrix)), sens_matrix.T), weights)
    delta = np.dot(delta, residual)
    # 更新假设标准
    hyp_std_conc += delta
    hyp_std_conc /= np.sum(hyp_std_conc)
    # 判断是否收敛
    if np.sum(np.abs(delta)) < tolerance:
        break
# 输出结果
print('假设标准浓度为：', hyp_std_conc)

In [ ]:
def moving_average(X, Y, window_size):
    """
    对输入的X和Y数组使用移动平均法进行处理，并返回处理后的Y数组和原始的X数组。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      window_size (int): 窗口大小，即取多少个数据点进行平均。

    Returns:
      新的Y和原来的X组成的元组。
    """
    # 初始化新的Y数组
    new_Y = np.zeros(len(Y))

    # 对每个数据点进行处理
    for i in range(len(Y)):
        # 计算窗口内数据的平均值
        start_idx = max(0, i - window_size // 2)
        end_idx = min(len(Y), i + window_size // 2 + 1)
        avg = np.mean(Y[start_idx:end_idx])

        # 保存处理后的数据
        new_Y[i] = avg

# 返回结果
return (new_Y, X)



In [ ]:
def mean_filter(X, Y, window_size):
    """
    对输入的X和Y数组使用均值滤波法进行处理，并返回处理后的Y数组和原始的X数组。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      window_size (int): 窗口大小，即取多少个数据点进行均值滤波。

    Returns:
      新的Y和原来的X组成的元组。
    """
    # 初始化新的Y数组
    new_Y = np.zeros(len(Y))

    # 对每个数据点进行处理
    for i in range(len(Y)):
        # 取出窗口内的数据，并计算其平均值
        start_idx = max(0, i - window_size // 2)
        end_idx = min(len(Y), i + window_size // 2 + 1)
        avg = np.mean(Y[start_idx:end_idx])

        # 保存处理后的数据
        new_Y[i] = avg

    # 返回结果
    return (new_Y, X)



In [ ]:

def weighted_moving_average(X, Y, window_size, weights):
    """
    对输入的X和Y数组使用加权移动平均法进行处理，并返回处理后的Y数组和原始的X数组。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      window_size (int): 窗口大小，即取多少个数据点进行权值平均。
      weights (numpy.ndarray): 权重数组，用于指定每个数据点的权重，长度应为window_size。

    Returns:
      新的Y和原来的X组成的元组。
    """
    # 初始化新的Y数组
    new_Y = np.zeros(len(Y))

    # 对每个数据点进行处理
    for i in range(len(Y)):
        # 计算窗口内数据的加权平均值
        start_idx = max(0, i - window_size // 2)
        end_idx = min(len(Y), i + window_size // 2 + 1)

        # 获取当前窗口的数据和对应的权重值
        window_data = Y[start_idx:end_idx]
        window_weights = weights[:len(window_data)]

        # 计算加权平均值
        avg = np.average(window_data, weights=window_weights)

        # 保存处理后的数据
        new_Y[i] = avg

    # 返回结果
    return (new_Y, X)



In [ ]:

from scipy.signal import savgol_filter

def sg_filter(X, Y, window_size, poly_order):
    """
    对输入的X和Y数组使用S-G滤波法进行处理，并返回处理后的Y数组和原始的X数组。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      window_size (int): 窗口大小，即用于进行滤波的点数。
      poly_order (int): 拟合多项式阶数。

    Returns:
      新的Y和原来的X组成的元组。
    """
    # 使用savgol_filter函数进行滤波
    new_Y = savgol_filter(Y, window_size, poly_order)

    # 返回结果
    return (new_Y, X)




In [ ]:

from pykalman import KalmanFilter

def kalman_filter(X, Y):
    """
    对输入的X和Y数组使用Kalman滤波法进行处理，并返回处理后的Y数组和原始的X数组。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。

    Returns:
      新的Y和原来的X组成的元组。
    """
    # 构建Kalman滤波器并使用观测值计算状态值
    kf = KalmanFilter(n_dim_obs=1, n_dim_state=1)
    states_pred = kf.em(Y).smooth(Y)[0]

    # 返回结果
    return (states_pred.ravel(), X)

、

In [ ]:

import numpy as np

def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用一次和二次导数法进行峰值识别，并返回峰值点在X轴上的位置以及Y轴上的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, 1]，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 计算X轴的步长, d(x) = x[i+1] - x[i]
    dx = np.gradient(X)

    # 求Y轴的一次、二次导数
    dy_first = np.gradient(Y, dx)
    dy_second = np.gradient(dy_first, dx)

    # 寻找peaks点
    peaks_indices, _ = find_peaks(Y, height=Y.mean() * threshold, distance=2*dx[0], prominence=(None, None))

    # 返回结果
    return (X[peaks_indices], Y[peaks_indices])



In [ ]:

import numpy as np
import pywt                  # 导入pywavelets库

def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用小波分析法进行峰值识别，
    并返回峰值点在X轴上的位置以及Y轴上对应的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, 1]，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 执行小波变换，过滤掉高频噪声信号
    cA, cD = pywt.dwt(Y, wavelet='haar')

    # 求cA的平均值std，作为判断峰值是否存在的阈值
    std = np.mean(cA*(cA>0))

    # 通过阈值来判断哪些峰值应该保留
    peaks_indices = pywt.find_peaks(cA, threshold=threshold*std)

    # 返回结果
    return (X[peaks_indices], Y[peaks_indices])



In [ ]:

def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用阈值匹配法进行峰值识别，
    并返回峰值点在X轴上的位置和对应的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, 1]，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 设置阈值，根据阈值选择保留哪些峰值点
    peaks_indices = np.where(Y > threshold * np.max(Y))

    # 返回结果
    return (X[peaks_indices], Y[peaks_indices])



In [ ]:


from scipy.optimize import curve_fit

def gauss(x, a, x0, sigma):
    """
    高斯函数定义。

    Args:
      x (numpy.ndarray): 自变量。
      a (float): 幅度因子。
      x0 (float): 中心频率。
      sigma (float): 带宽(sigma越大，分布越广)。

    Returns:
      该组参数下的高斯函数结果。
    """
    return a * np.exp(-(x - x0)**2 / (2*sigma**2))

def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用高斯函数拟合法进行峰值识别，
    并返回峰值点在X轴上的位置和对应的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, 1]，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 进行高斯函数拟合
    popt, _ = curve_fit(gauss, X, Y)

    # 根据阈值，选择需要保留的谷底点
    peaks_indices = np.where(Y > threshold * gauss(X, *popt).max())

    # 返回结果
    return (X[peaks_indices], Y[peaks_indices])



In [ ]:


def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用匹配追踪法进行峰值识别，
    并返回峰值点在X轴上的位置和对应的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, 1]，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 获取光谱数据的长度和峰值数量区间
    n_points = len(Y)
    n_windows = int(n_points/10)

    # 初始化一个空列表用于保存峰值信息
    peaks_indices = []

    # 进行匹配追踪
    for i in range(n_windows):
        window = Y[i*10:(i+1)*10]
        max_idx = i*10 + np.argmax(window)
        max_val = Y[max_idx]
        if max_val > threshold * np.max(Y):
            peaks_indices.append(max_idx)

    # 返回结果
    return (X[peaks_indices], Y[peaks_indices])



In [ ]:


def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用峰值最大阈值法进行峰值识别，
    并返回峰值点在X轴上的位置和对应的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, 1]，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 使用argrelextrema函数找到Y的峰值点位置
    max_indices = argrelextrema(Y, np.greater)[0]

    # 根据阈值，选择需要保留的谷底点
    peaks_indices = max_indices[np.where(Y[max_indices] > threshold * np.max(Y))]

    # 返回结果
    return (X[peaks_indices], Y[peaks_indices])



In [ ]:

def find_peaks(X, Y, threshold=0.1):
    """
    对输入的X和Y数组使用峰面积阈值法进行峰值识别，
    并返回峰值点在X轴上的位置和对应的值。

    Args:
      X (numpy.ndarray): 包含光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含光谱探测器计数数量大小的Y轴数组。
      threshold (float): 用于筛选峰值的阈值，取值范围为(0, inf)，默认值为0.1。

    Returns:
      峰值点在X轴上的位置和Y轴上的值组成的元组。
    """
    # 使用peak_widths函数找到所有峰值
    peak_indexes, _, _ = peak_widths(Y, peaks=None, rel_height=(threshold, None))

    # 计算每个峰值的面积
    peak_areas = []
    for index in peak_indexes:
        start_index = int(round(index - 0.5))
        end_index = int(round(index + 0.5))
        y_slice = Y[start_index:(end_index+1)]
        x_slice = X[start_index:(end_index+1)]
        area = np.trapz(y_slice, x_slice)
        peak_areas.append(area)

    # 根据面积排序，返回前n个
    n_peaks = len(peak_indexes)
    peak_areas = np.array(peak_areas)
    sorted_indices = np.argsort(-peak_areas)
    n_returned = min(n_peaks, sum(peak_areas > 0))
    returned_indices = peak_indexes[sorted_indices[:n_returned]]
    return (X[returned_indices], Y[returned_indices])



In [ ]:

import pywt

def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用小波变换法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组。

    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。

    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    # 使用小波变换对数据进行降噪和分解
    coeffs = pywt.swt(Y, 'db2', level=4)

    # 根据峰值位置分割出不同峰值范围的信号
    separated_Y = []
    for i in range(len(peaks)-1):
        start_index = np.searchsorted(X, peaks[i][0], side='left')
        end_index = np.searchsorted(X, peaks[i+1][0], side='right')
        signal = coeffs[-1][start_index:end_index]
        for j in range(2,6):
            signal = np.concatenate([coeffs[-j][start_index:end_index],signal])
        # 重新合成信号
        separated_Y.append(pywt.iswt(signal, 'db2'))

    # 处理最后一个峰范围的信号
    start_index = np.searchsorted(X, peaks[-1][0], side='left')
    end_index = len(Y)
    signal = coeffs[-1][start_index:end_index]
    for j in range(2,6):
        signal = np.concatenate([coeffs[-j][start_index:end_index],signal])
    # 重新合成信号
    separated_Y.append(pywt.iswt(signal, 'db2'))

    # 合并分离后的信号，得到最终结果
    smoothed_Y = np.concatenate(separated_Y)
    return (X, smoothed_Y)



In [ ]:


import numpy as np
from scipy.fft import fft, ifft
def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用傅里叶变换法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组。
    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。
    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    T = X[1] - X[0]
    # 对全部信号进行傅里叶变换
    freq_signal = fft(Y)
    # 根据峰值在X轴的位置找到其对应的频率范围
    peak_freq_ranges = []
    for i in range(len(peaks)-1):
        start_index = np.searchsorted(X, peaks[i][0], side='left')
        end_index = np.searchsorted(X, peaks[i+1][0], side='right')
        peak_freq_ranges.append((start_index / len(X) / T, end_index / len(X) / T))
    # 处理最后一个峰范围的信号
    start_index = np.searchsorted(X, peaks[-1][0], side='left')
    end_index = len(Y)
    peak_freq_ranges.append((start_index / len(X) / T, end_index / len(X) / T))
    # 根据频率范围将不同峰值所在的信号进行+分离处理
    separated_signals = []
    for freq_range in peak_freq_ranges:
        min_freq = freq_range[0]
        max_freq = freq_range[1]
        num_samples = len(Y)
        passband = np.logical_and(np.abs(np.linspace(-0.5, 0.5, num_samples)) >= min_freq,
                                  np.abs(np.linspace(-0.5, 0.5, num_samples)) <= max_freq)
        stopband = ~passband
        # 设计滤波器，这里使用了FIR窗口方法
        fc = (min_freq + max_freq) / 2
        window_length = 101
        taps = np.sinc(2 * fc * (np.arange(window_length) - (window_length-1)/2))
        taps *= np.hamming(window_length)
        taps /= np.sum(taps)
        # 应用滤波器并反演频域得到时域信号
        filtered_freq_signal = freq_signal * passband.astype(float)
        filtered_signal = ifft(filtered_freq_signal)
        filtered_signal *= np.exp(2j * np.pi * fc * np.arange(num_samples) * T)
        separated_signals.append(filtered_signal)
    # 合并分离后的信号，得到最终结果
    smoothed_Y = np.sum(separated_signals, axis=0)
    return (X, np.real(smoothed_Y))


In [ ]:

import numpy as np
from scipy.optimize import curve_fit

def gaussian(x, a, b, c):
    """高斯分布函数"""
    return a * np.exp(-(x - b)**2 / (2 * c**2))

def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用高斯拟合法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组。

    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。

    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    X = np.asarray(X)
    Y = np.asarray(Y)

    # 对每个峰值使用高斯拟合函数并分离处理
    smoothed_Y = np.zeros_like(Y)
    for peak in peaks:
        x_range = np.logical_and(X > peak[0]-1, X < peak[0]+1)  # 取出拟合区间
        xdata = X[x_range]
        ydata = Y[x_range]
        p0 = [ydata.max(), peak[0], 0.2]  # 设置初始拟合参数
        popt, _ = curve_fit(gaussian, xdata, ydata, p0=p0,
                            bounds=([0, peak[0]-0.5, 0], [np.inf, peak[0]+0.5, 0.5]))  # 进行高斯拟合
        smoothed_Y += gaussian(X, *popt)  # 将拟合结果累加到最终光谱上

    return X, smoothed_Y



In [ ]:

import numpy as np
def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用偏导数法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组。

    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。

    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    smoothed_Y = np.zeros_like(Y)
    # 分别对每个峰值所在的区间进行偏导数处理，最终得到分离后的曲线
    for i in range(len(peaks)-1):
        left_peak_x, _ = peaks[i]
        right_peak_x, _ = peaks[i+1]
        # 取出该区间内的数据以及左右两端一段固定长度区间内的数据
        left_index = np.searchsorted(X, left_peak_x-10)
        right_index = np.searchsorted(X, right_peak_x+10)
        xdata = X[left_index:right_index]
        ydata = Y[left_index:right_index]
        # 对该区间内的数据进行偏导数处理并找到响应最强的谷值所在位置
        num_iterations = 10
        lambda_ = 0.2
        for j in range(num_iterations):
            dydx = np.gradient(ydata, xdata)
            ydata += lambda_ * np.abs(dydx) * np.sign(dydx)
        min_index = left_index + np.argmin(ydata[left_index-xdata[0]:right_index-xdata[0]])
        # 将该峰区间内最近的两个谷值作为分割点，对两个子区间进行二次多项式拟合，得到分离曲线
        if i == 0:
            left_seg_start = left_index
        else:
            left_seg_start = right_last_min_index
        seg_end = min_index-5
        c_left = np.polyfit(X[left_seg_start:seg_end], Y[left_seg_start:seg_end], 2)
        left_curve = np.polyval(c_left, X[left_seg_start:seg_end])
        seg_start = min_index+5
        if i < len(peaks)-2:
            right_seg_end = right_index
        else:
            right_seg_end = np.searchsorted(X, peaks[-1][0]-10)
        c_right = np.polyfit(X[seg_start:right_seg_end], Y[seg_start:right_seg_end], 2)
        right_curve = np.polyval(c_right, X[seg_start:right_seg_end])
        smoothed_Y[left_seg_start:seg_end] += left_curve
        smoothed_Y[seg_start:right_seg_end] += right_curve
        right_last_min_index = seg_start + np.argmin(Y[seg_start-right_index:right_seg_end-right_index])
    return (X, smoothed_Y)




In [ ]:

import numpy as np
import pymc3 as pm

def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用贝叶斯反演法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组。

    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。

    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    with pm.Model() as model:  # 定义模型
        variables = []
        # 构造信号分量对应的贝叶斯变量并添加到模型中
        for peak in peaks:
            peak_x, _= peak
            amplitude = pm.Uniform('amplitude_{}'.format(peak_x), 0, Y.max())
            center = pm.Uniform('center_{}'.format(peak_x), peak_x-1, peak_x+1)
            width = pm.Uniform('width_{}'.format(peak_x), 0, 2)

            variables.append((amplitude, center, width))

        # 构造背景分量对应的贝叶斯变量并添加到模型中
        bkg = pm.Uniform('background', 0, Y.max())

        # 定义信号和背景的线性组合模型，得到预测的谱线
        peaks_predicted = np.sum([v[0]*np.exp(-0.5*((X-v[1])/v[2])**2) for v in variables], axis=0)
        y_predicted = peaks_predicted + bkg

        # 定义似然函数，即残差平方和，用于拟合数据
        likelihood = pm.Poisson('likelihood', mu=y_predicted, observed=Y)

        # 使用MCMC方法进行贝叶斯反演，得到最终的分离结果
        trace = pm.sample(10000, tune=5000, chains=4)

    smoothed_Y = np.zeros_like(Y)
    for variable_set in trace.get_values(variables):
        peaks_predicted = np.sum([v[0]*np.exp(-0.5*((X-v[1])/v[2])**2) for v in variable_set], axis=0)
        smoothed_Y += peaks_predicted

    return X, smoothed_Y/len(trace.get_values(variables))



In [2]:

import numpy as np
from scipy.optimize import curve_fit

def gaussian(x, amplitude, center, width):
    """
    定义高斯峰函数。
    """
    return amplitude * np.exp(-0.5*((x-center)/width)**2)

def multi_gaussian(x, *params):
    """
    多个高斯峰的线性组合函数，
    其中参数列表params包含每个峰的幅值、中心位置和宽度等参数。
    """
    n_peaks = len(params)//3
    peaks = [gaussian(x, params[i], params[i+1], params[i+2]) for i in range(0, n_peaks*3, 3)]
    return np.sum(peaks, axis=0)

def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用线性组合法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组组成的元组。

    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。

    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    n_peaks = len(peaks)
    guess_params = [y for _, y in peaks] + [1]*(3*n_peaks)

    # 使用拟合函数和初始猜测参数对光谱进行拟合，得到每个峰的幅值、中心位置和宽度等参数
    fit_params, _ = curve_fit(multi_gaussian, X, Y, p0=guess_params)
    smoothed_Y = multi_gaussian(X, *fit_params[:n_peaks*3])  # 计算拟合后的光谱曲线

    return X, smoothed_Y


SyntaxError: invalid syntax (820394078.py, line 39)

In [ ]:

import numpy as np
from scipy.signal import find_peaks

def separate_peaks(X, Y, peaks):
    """
    对输入的X和Y数组使用特征谱峰法分离谱峰，
    并返回分离谱峰后的X和Y轴上的数组组成的元组。

    Args:
      X (numpy.ndarray): 包含所有光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含所有光谱探测器计数数量大小的Y轴数组。
      peaks (list of tuple): 峰值点在X轴和Y轴上的位置组成的元组列表。

    Returns:
      分离谱峰后的X和Y轴上的数组组成的元组。
    """
    peak_indexes, _ = find_peaks(Y)  # 找到所有极大值（峰）
    smoothed_Y = np.zeros_like(Y)
    for index in peak_indexes:
        _, y_peak = peaks[np.argmin([abs(index-x) for x, _ in peaks])]  # 找到最接近该峰的人工指定峰
        window = slice(max(0, index-10), min(Y.size, index+10))  # 构造固定宽度的窗口
        shifted_X = X[window] - X[index]
        shifted_Y = Y[window] / y_peak
        weights = np.exp(-((shifted_X/5)**2 + (shifted_Y-1)**2))  # 以该峰为中心，给窗口内每个点分配权重
        smoothed_Y[window] += weights * Y[index]  # 将该峰加权后的光谱贡献加入到拟合结果中

    return X, smoothed_Y



In [ ]:
import numpy as np
from scipy.interpolate import interp1d

def fit_subtract(X, Y, order=2, num_points=1000):
    """
    使用多项式拟合法对光谱数据Y进行背景扣除。

    Args:
      X (numpy.ndarray): 包含该光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含该光谱探测器计数数量大小的Y轴数组。
      order (int): 拟合多项式的阶数，默认为2。
      num_points (int): 插值函数使用的点数，默认为1000。

    Returns:
      扣除背景后的X和Y轴上的数组组成的元组。
    """
    # 首先将数据转换为插值函数，并在较小的范围内进行插值
    f = interp1d(X, Y, kind='cubic')
    x_new = np.linspace(np.min(X), np.max(X), num_points)
    y_new = f(x_new)

    # 利用多项式拟合对插值曲线进行近似
    p = np.polyfit(x_new, y_new, order)
    y_fit = np.polyval(p, X)

    # 从初始光谱中扣除拟合曲线得到背景较纯净的光谱
    Y_subtracted = Y - y_fit

    # 返回扣除背景后的X和Y轴上的数组组成的元组
    return X, Y_subtracted

In [ ]:

import numpy as np
from sklearn.decomposition import PCA

def pca_subtract(X, Y, n_components=10):
    """
    使用主成分分析法对光谱数据Y进行背景扣除。

    Args:
      X (numpy.ndarray): 包含该光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含该光谱探测器计数数量大小的Y轴数组。
      n_components (int): 需要保留的主成分个数，默认为10个。

    Returns:
      扣除背景后的X和Y轴上的数组组成的元组。
    """
    # 对光谱数据做标准化
    mean = np.mean(Y)
    std = np.std(Y)
    Y_norm = (Y - mean) / std

    # 使用 PCA 分析并选择前 n 个主成分
    pca = PCA(n_components=n_components)
    pca.fit(Y_norm.T)
    pcs = pca.components_

    # 将选中的主成分对输入数据做逆变换，得到背景光谱
    compts = pca.transform(Y_norm.T)
    bg_pca = pca.inverse_transform(compts).T * std + mean

    # 从初始光谱中扣除背景光谱得到扣除背景后的光谱
    Y_subtracted = Y - bg_pca

    # 返回扣除背景后的X和Y轴上的数组组成的元组
    return X, Y_subtracted




In [ ]:

import numpy as np

def adaptive_subtract(X, Y, max_iter=100, order=1, alpha=0.01):
    """
    使用自适应背景扣除法对光谱数据Y进行背景扣除。

    Args:
      X (numpy.ndarray): 包含该光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含该光谱探测器计数数量大小的Y轴数组。
      max_iter (int): 迭代次数的上限，默认为100。
      order (int): 多项式拟合的阶数，默认为1。
      alpha (float): 步长，默认为0.01。

    Returns:
      扣除背景后的X和Y轴上的数组组成的元组。
    """
    # 初始化权重向量 w
    w = np.ones(Y.shape[0])

    # 多项式拟合处理得到候选背景信号
    p = np.polyfit(X, Y, order)
    bg_estimate = np.polyval(p, X)

    # 对候选背景信号进行迭代优化
    for i in range(max_iter):
        num = np.sum((w ** 2) * (Y - bg_estimate) * Y)
        den = np.sum((w ** 2) * (bg_estimate ** 2))
        a = num / den

        bg_estimate = a * bg_estimate + (1 - a) * Y

        w = np.exp(-(Y - bg_estimate) ** 2 / np.var(Y - bg_estimate))

    # 将扣除后的背景从原始信号中减去
    Y_subtracted = Y - bg_estimate

    # 返回扣除背景后的X和Y轴上的数组组成的元组
    return X, Y_subtracted



In [ ]:


import numpy as np
from scipy.signal import find_peaks
def estimate_lines(X, Y, sigma=3, threshold=0.1):
    """
    使用估计谱线法对光谱数据Y进行峰检测和拟合。

    Args:
      X (numpy.ndarray): 包含该光谱能量通道的X轴数组。
      Y (numpy.ndarray): 包含该光谱探测器计数数量大小的Y轴数组。
      sigma (float): 计算局部区间峰值均值和标准差所用的高斯核函数跨度，默认为3个数据点。
      threshold (float): 相对于最大值的幅度阈值，默认为0.1。

    Returns:
      拟合出的峰线数组（包含位置、幅度和半高全宽等信息）。
    """
    # 首先通过高斯滤波平滑整个谱线，得到更好的特征信息
    smoothed = np.convolve(Y, np.ones(sigma)/sigma, mode='same')
    
    # 利用 SciPy 的峰检测函数找到局部区间峰值的位置
    peaks, _ = find_peaks(smoothed, height=threshold*np.max(smoothed))
    # 初始化峰线数组，用于储存峰位置、幅度和半高全宽等信息
    lines = np.zeros((len(peaks), 3))
    # 针对每一个局部峰值进行光谱线拟合
    for i, peak in enumerate(peaks):
        half_max = smoothed[peak] / 2.
        # 利用多项式拟合法从背景中提取出当前峰的信号
        bg = np.polyval(np.polyfit(X[np.abs(X-X[peak]) > 50], Y[np.abs(X-X[peak]) > 50], deg=6), X)
        y1 = Y - bg
        y1[y1 < 0.] = 0.

        line_half, *_ = find_peaks(y1[peak-10:peak+10], half_max, width=5)
        if len(line_half) == 1:
            lines[i, 0] = X[peak]
            lines[i, 1] = Y[peak] - bg[peak]
            left_idx = np.where(Y[:peak] - bg[:peak] > half_max)[0][-1]
            right_idx = np.where(Y[peak:] - bg[peak:] < half_max)[0][0] + peak
            lines[i, 2] = X[right_idx] - X[left_idx]
return lines

